# Basline

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, logging
from datasets import load_dataset
import numpy as np
import copy
import time
from collections import defaultdict
import warnings
import gc
import sys
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# -----------------------------------------------------------------------------
# 1. CONFIGURATION
# -----------------------------------------------------------------------------

warnings.filterwarnings('ignore')
logging.set_verbosity_error()

@dataclass
class ExperimentConfig:
    MODEL_NAME: str = "distilbert-base-uncased"
    DEVICE: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BATCH_SIZE: int = 16
    MAX_LENGTH: int = 128
    LEARNING_RATE: float = 2e-5
    FINETUNE_EPOCHS: int = 3
    RETAIN_SAMPLES_PER_CLASS: int = 200
    FORGET_SCENARIOS: List[int] = None
    SEEDS: List[int] = None
    NUM_CLASSES: int = 4
    
    def __post_init__(self):
        if self.SEEDS is None: self.SEEDS = [42, 123, 456]
        if self.FORGET_SCENARIOS is None: self.FORGET_SCENARIOS = [50, 100, 200]

# -----------------------------------------------------------------------------
# 2. UTILS & STYLING (Q1 Paper Formatting)
# -----------------------------------------------------------------------------

class ReportStyler:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

    @staticmethod
    def print_banner(text):
        print(f"\n{ReportStyler.BOLD}{'='*140}{ReportStyler.ENDC}")
        print(f"{ReportStyler.BOLD}{ReportStyler.HEADER}  {text.upper()}{ReportStyler.ENDC}")
        print(f"{ReportStyler.BOLD}{'='*140}{ReportStyler.ENDC}")

    @staticmethod
    def print_scenario_header(cls_name, n, seed):
        print(f"\n{ReportStyler.CYAN}┌── SCENARIO: Forget '{cls_name}' | N={n} | Seed={seed} {'─'*80}{ReportStyler.ENDC}")

    @staticmethod
    def print_baseline_row(name, f_acc, r_acc, all_acc, ece, mia_auc):
        print(f"│ {ReportStyler.YELLOW}{name:<12}{ReportStyler.ENDC} │ F-Acc: {f_acc:.3f} │ R-Acc: {r_acc:.3f} │ All-Acc: {all_acc:.3f} │ ECE: {ece:.3f} │ MIA: {mia_auc:.3f}")

    @staticmethod
    def print_table_header():
        print(f"│ {'─'*138}")
        print(f"│ {ReportStyler.BOLD}{'METHOD':<10} │ {'F-ACC':<8} │ {'R-ACC':<8} │ {'ALL-ACC':<8} │ {'ECE':<8} │ {'MIA-AUC':<8} │ {'PRIVACY':<8} │ {'H-SCORE':<8} │ {'TIME':<7}{ReportStyler.ENDC}")
        print(f"│ {'─'*138}")

    @staticmethod
    def print_result_row(method, f_acc, r_acc, all_acc, ece, mia, priv, h_score, time):
        # Color coding for quick visual analysis
        c_f = ReportStyler.GREEN if f_acc < 0.05 else ""
        c_r = ReportStyler.FAIL if r_acc < 0.7 else ""
        c_h = ReportStyler.BOLD if h_score > 0.85 else ""
        
        print(f"│ {method:<10} │ {c_f}{f_acc:<8.3f}{ReportStyler.ENDC} │ {c_r}{r_acc:<8.3f}{ReportStyler.ENDC} │ {all_acc:<8.3f} │ {ece:<8.3f} │ {mia:<8.3f} │ {priv:<8.3f} │ {c_h}{h_score:<8.3f}{ReportStyler.ENDC} │ {time:<7.1f}")

def set_seed(seed: int):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def clear_memory():
    gc.collect(); torch.cuda.empty_cache()

def safe_deepcopy_model(model):
    """Deepcopy via CPU to prevent VRAM spikes."""
    clear_memory()
    device = next(model.parameters()).device
    model_cpu = model.to('cpu')
    model_copy = copy.deepcopy(model_cpu)
    model.to(device) # Restore original
    return model_copy

# -----------------------------------------------------------------------------
# 3. DATA & EVALUATION
# -----------------------------------------------------------------------------

class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
        self.labels = labels
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx):
        return {'input_ids': self.encodings['input_ids'][idx], 
                'attention_mask': self.encodings['attention_mask'][idx], 
                'labels': torch.tensor(self.labels[idx], dtype=torch.long)}

class Evaluator:
    def __init__(self, device, config):
        self.device = device
        self.config = config
        
    def get_full_metrics(self, model, loaders, forget_class):
        """Returns comprehensive dictionary of all metrics for the paper"""
        model.eval()
        model.to(self.device)
        
        # 1. Classification Metrics (Accuracy, ECE)
        all_preds, all_labels, all_probs = [], [], []
        with torch.no_grad():
            for batch in loaders['test_all']:
                out = model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device))
                probs = F.softmax(out.logits, dim=-1)
                all_preds.extend(torch.argmax(probs, dim=-1).cpu().numpy())
                all_labels.extend(batch['labels'].cpu().numpy())
                all_probs.extend(probs.cpu().numpy())
        
        all_preds, all_labels, all_probs = np.array(all_preds), np.array(all_labels), np.array(all_probs)
        
        # Per Class Accuracy
        acc_per_class = {}
        for c in range(self.config.NUM_CLASSES):
            mask = all_labels == c
            acc_per_class[c] = accuracy_score(all_labels[mask], all_preds[mask]) if mask.sum() > 0 else 0.0

        # ECE Calculation
        confidences = np.max(all_probs, axis=1)
        predictions = np.argmax(all_probs, axis=1)
        accuracies = (predictions == all_labels)
        bins = np.linspace(0, 1, 11)
        ece = 0.0
        for i in range(10):
            mask = (confidences >= bins[i]) & (confidences < bins[i+1])
            if mask.sum() > 0:
                ece += mask.sum()/len(all_labels) * abs(accuracies[mask].mean() - confidences[mask].mean())

        # 2. Privacy Metrics (MIA)
        # Using Loss-Gap Attack
        crit = nn.CrossEntropyLoss(reduction='none')
        f_losses, r_losses = [], []
        with torch.no_grad():
            for batch in loaders['forget_train']: # Members
                out = model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device))
                f_losses.extend(crit(out.logits, batch['labels'].to(self.device)).cpu().numpy())
            for batch in loaders['retain_test']: # Non-Members (approx)
                out = model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device))
                r_losses.extend(crit(out.logits, batch['labels'].to(self.device)).cpu().numpy())
        
        # Member = 1 (Low Loss), Non-Member = 0 (High Loss)
        # We invert loss for AUC calculation so higher score = likely member
        y_true = np.concatenate([np.ones(len(f_losses)), np.zeros(len(r_losses))])
        y_score = np.concatenate([-np.array(f_losses), -np.array(r_losses)])
        try: mia_auc = roc_auc_score(y_true, y_score)
        except: mia_auc = 0.5
        
        # Privacy Score: 1.0 is perfect privacy (AUC=0.5), 0.0 is total leakage (AUC=1.0 or 0.0)
        privacy_score = 1.0 - (2 * abs(mia_auc - 0.5))

        # 3. Composite Scores
        f_acc = acc_per_class[forget_class]
        r_acc = np.mean([acc_per_class[c] for c in range(4) if c != forget_class])
        
        # Efficacy: How close is F-Acc to 0? (1 - F_Acc)
        eff = 1.0 - f_acc
        # Preservation: Just R-Acc
        pres = r_acc
        # Harmonic Mean Score (The "Paper Score")
        h_score = 2 * (eff * pres) / (eff + pres + 1e-6)

        return {
            'f_acc': f_acc,
            'r_acc': r_acc,
            'all_acc': accuracy_score(all_labels, all_preds),
            'ece': ece,
            'mia_auc': mia_auc,
            'privacy_score': privacy_score,
            'h_score': h_score
        }

# -----------------------------------------------------------------------------
# 4. UNLEARNING METHODS
# -----------------------------------------------------------------------------

class BaseUnlearner:
    def __init__(self, model, device, loaders, config):
        self.model = safe_deepcopy_model(model).to(device)
        self.device = device
        self.loaders = loaders
        self.config = config
        self.ref_model = safe_deepcopy_model(model).to(device)
        self.ref_model.eval()

class GradientAscent(BaseUnlearner):
    def run(self):
        opt = torch.optim.AdamW(self.model.parameters(), lr=1e-5)
        self.model.train()
        for _ in range(5): # 5 Epochs
            for batch in self.loaders['forget_train']:
                out = self.model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device), labels=batch['labels'].to(self.device))
                loss = out.loss
                opt.zero_grad(); (-loss).backward(); opt.step() # Maximize loss
            # Repair step on retain data
            for batch in self.loaders['retain_train']:
                out = self.model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device), labels=batch['labels'].to(self.device))
                opt.zero_grad(); out.loss.backward(); opt.step()
        return self.model

class NPO(BaseUnlearner):
    def run(self):
        opt = torch.optim.AdamW(self.model.parameters(), lr=1e-5)
        self.model.train()
        beta = 0.1
        for _ in range(5):
            for batch in self.loaders['forget_train']:
                ids, mask, lbls = batch['input_ids'].to(self.device), batch['attention_mask'].to(self.device), batch['labels'].to(self.device)
                out = self.model(ids, attention_mask=mask, labels=lbls)
                with torch.no_grad(): ref_out = self.ref_model(ids, attention_mask=mask, labels=lbls)
                loss = -F.logsigmoid(-beta * (ref_out.loss - out.loss))
                opt.zero_grad(); loss.backward(); opt.step()
            for batch in self.loaders['retain_train']:
                out = self.model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device), labels=batch['labels'].to(self.device))
                opt.zero_grad(); out.loss.backward(); opt.step()
        return self.model

class TaskVector(BaseUnlearner):
    def run(self):
        # We need the pre-trained (zero-shot) model for this
        # Assuming we can instantiate a fresh one or pass it. 
        # For simplicity in this script, we assume 'ref_model' is the Fine-tuned one.
        # We will simulate a "Pre-trained" base by loading fresh.
        base_model = AutoModelForSequenceClassification.from_pretrained(self.config.MODEL_NAME, num_labels=4).to(self.device)
        
        # W_new = W_pretrained + (W_finetuned - W_pretrained) - alpha * (W_finetuned - W_pretrained)
        # Ideally: W_new = W_finetuned - alpha * (W_finetuned - W_pretrained)
        alpha = 0.4
        
        with torch.no_grad():
            for p_new, p_ft, p_base in zip(self.model.parameters(), self.ref_model.parameters(), base_model.parameters()):
                task_vec = p_ft.data - p_base.data
                p_new.data = p_ft.data - (alpha * task_vec)
        return self.model

class Relabel(BaseUnlearner):
    def run(self):
        opt = torch.optim.AdamW(self.model.parameters(), lr=1e-5)
        self.model.train()
        for _ in range(3):
            for batch in self.loaders['forget_train']:
                ids, mask, lbls = batch['input_ids'].to(self.device), batch['attention_mask'].to(self.device), batch['labels'].to(self.device)
                # Random labels
                rand_lbls = torch.randint(0, 4, lbls.shape).to(self.device)
                out = self.model(ids, attention_mask=mask, labels=rand_lbls)
                opt.zero_grad(); out.loss.backward(); opt.step()
            for batch in self.loaders['retain_train']:
                out = self.model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device), labels=batch['labels'].to(self.device))
                opt.zero_grad(); out.loss.backward(); opt.step()
        return self.model

class Prune(BaseUnlearner):
    def run(self):
        # Selective pruning based on activation difference
        self.model.eval()
        base = self.model.distilbert.transformer
        
        for layer in base.layer:
            # Hook intermediate activations (lin1 output)
            f_acts, r_acts = [], []
            h1 = layer.ffn.lin1.register_forward_hook(lambda m,i,o: f_acts.append(o.mean(dim=(0,1)).detach().cpu()))
            with torch.no_grad():
                for i, b in enumerate(self.loaders['forget_train']):
                    if i>5: break
                    self.model(b['input_ids'].to(self.device), attention_mask=b['attention_mask'].to(self.device))
            h1.remove()
            
            h2 = layer.ffn.lin1.register_forward_hook(lambda m,i,o: r_acts.append(o.mean(dim=(0,1)).detach().cpu()))
            with torch.no_grad():
                for i, b in enumerate(self.loaders['retain_train']):
                    if i>5: break
                    self.model(b['input_ids'].to(self.device), attention_mask=b['attention_mask'].to(self.device))
            h2.remove()

            if f_acts and r_acts:
                # Neurons that activate more for Forget than Retain
                diff = torch.stack(f_acts).mean(0) - torch.stack(r_acts).mean(0)
                # Prune top 10%
                thresh = torch.quantile(diff, 0.90)
                mask = (diff < thresh).float().to(self.device)
                
                # Apply mask to weights
                with torch.no_grad():
                    layer.ffn.lin1.weight.data *= mask.unsqueeze(1)
                    layer.ffn.lin1.bias.data *= mask
                    layer.ffn.lin2.weight.data *= mask.unsqueeze(0)
        
        # Quick repair
        opt = torch.optim.AdamW(self.model.parameters(), lr=1e-5)
        self.model.train()
        for _ in range(1):
             for batch in self.loaders['retain_train']:
                out = self.model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device), labels=batch['labels'].to(self.device))
                opt.zero_grad(); out.loss.backward(); opt.step()
        return self.model

# -----------------------------------------------------------------------------
# 5. MAIN EXECUTION
# -----------------------------------------------------------------------------

def load_data(config, tokenizer):
    raw = load_dataset("ag_news")
    data = defaultdict(lambda: defaultdict(list))
    for split in ['train', 'test']:
        for item in raw[split]:
            data[split][item['label']].append(item['text'])
    return data

def make_loader(texts, labels, tokenizer, batch_size, shuffle=False):
    ds = TextClassificationDataset(texts, labels, tokenizer, 128)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

def run_experiment():
    conf = ExperimentConfig()
    ReportStyler.print_banner("Q1 Unlearning Benchmark: AG News")
    
    tokenizer = AutoTokenizer.from_pretrained(conf.MODEL_NAME)
    raw_data = load_data(conf, tokenizer)
    evaluator = Evaluator(conf.DEVICE, conf)

    final_results = []

    for seed in conf.SEEDS:
        set_seed(seed)
        
        for f_cls in range(conf.NUM_CLASSES): # Iterate all classes
            class_names = ["World", "Sports", "Business", "Sci/Tech"]
            target_name = class_names[f_cls]
            
            for n_forget in conf.FORGET_SCENARIOS:
                ReportStyler.print_scenario_header(target_name, n_forget, seed)
                
                # --- Prepare Data ---
                # Forget Train
                f_train_txt = raw_data['train'][f_cls][:n_forget]
                f_train_lbl = [f_cls]*n_forget
                
                # Retain Train
                r_train_txt, r_train_lbl = [], []
                for c in range(4):
                    if c != f_cls:
                        r_train_txt.extend(raw_data['train'][c][:conf.RETAIN_SAMPLES_PER_CLASS])
                        r_train_lbl.extend([c]*conf.RETAIN_SAMPLES_PER_CLASS)
                
                # Test Sets
                t_f_txt = raw_data['test'][f_cls][:200]
                t_f_lbl = [f_cls]*200
                t_r_txt, t_r_lbl = [], []
                for c in range(4):
                    if c != f_cls:
                        t_r_txt.extend(raw_data['test'][c][:200])
                        t_r_lbl.extend([c]*200)

                loaders = {
                    'forget_train': make_loader(f_train_txt, f_train_lbl, tokenizer, conf.BATCH_SIZE, True),
                    'retain_train': make_loader(r_train_txt, r_train_lbl, tokenizer, conf.BATCH_SIZE, True),
                    'forget_test':  make_loader(t_f_txt, t_f_lbl, tokenizer, conf.BATCH_SIZE, False),
                    'retain_test':  make_loader(t_r_txt, t_r_lbl, tokenizer, conf.BATCH_SIZE, False),
                    'test_all':     make_loader(t_f_txt+t_r_txt, t_f_lbl+t_r_lbl, tokenizer, conf.BATCH_SIZE, False)
                }

                # --- 1. Pre-trained Baseline ---
                model = AutoModelForSequenceClassification.from_pretrained(conf.MODEL_NAME, num_labels=4).to(conf.DEVICE)
                base_metrics = evaluator.get_full_metrics(model, loaders, f_cls)
                ReportStyler.print_baseline_row("PRE-TRAINED", base_metrics['f_acc'], base_metrics['r_acc'], base_metrics['all_acc'], base_metrics['ece'], base_metrics['mia_auc'])

                # --- 2. Fine-tuning ---
                model.train()
                opt = torch.optim.AdamW(model.parameters(), lr=conf.LEARNING_RATE)
                # Combine loaders for stable finetuning
                combined = ConcatDataset([loaders['forget_train'].dataset, loaders['retain_train'].dataset])
                train_l = DataLoader(combined, batch_size=conf.BATCH_SIZE, shuffle=True)
                
                for _ in range(conf.FINETUNE_EPOCHS):
                    for batch in train_l:
                        out = model(batch['input_ids'].to(conf.DEVICE), attention_mask=batch['attention_mask'].to(conf.DEVICE), labels=batch['labels'].to(conf.DEVICE))
                        opt.zero_grad(); out.loss.backward(); opt.step()
                
                ft_metrics = evaluator.get_full_metrics(model, loaders, f_cls)
                ReportStyler.print_baseline_row("FINE-TUNED", ft_metrics['f_acc'], ft_metrics['r_acc'], ft_metrics['all_acc'], ft_metrics['ece'], ft_metrics['mia_auc'])
                
                # Check if finetuning worked
                if ft_metrics['f_acc'] < 0.7:
                    print(f"{ReportStyler.FAIL}Skipping: Model failed to learn target class.{ReportStyler.ENDC}")
                    continue

                # --- 3. Unlearning Loop ---
                ReportStyler.print_table_header()
                
                methods = [
                    ("GA+GD", GradientAscent),
                    ("NPO", NPO),
                    ("TASKVEC", TaskVector),
                    ("PRUNE", Prune),
                    ("RELABEL", Relabel)
                ]

                # We must use a CPU copy of the FT model as the starting point for every method
                ft_model_cpu = safe_deepcopy_model(model).to('cpu')

                for name, Cls in methods:
                    try:
                        # Restore fresh copy
                        current_model = safe_deepcopy_model(ft_model_cpu).to(conf.DEVICE)
                        
                        t0 = time.time()
                        unlearner = Cls(current_model, conf.DEVICE, loaders, conf)
                        unlearned_model = unlearner.run()
                        dur = time.time() - t0
                        
                        res = evaluator.get_full_metrics(unlearned_model, loaders, f_cls)
                        
                        ReportStyler.print_result_row(
                            name, res['f_acc'], res['r_acc'], res['all_acc'], 
                            res['ece'], res['mia_auc'], res['privacy_score'], 
                            res['h_score'], dur
                        )
                        
                        final_results.append({
                            'seed': seed, 'n': n_forget, 'class': target_name, 'method': name, **res
                        })
                        
                        del unlearned_model, unlearner
                        clear_memory()
                        
                    except Exception as e:
                         print(f"│ {ReportStyler.FAIL}{name:<10} │ ERROR: {str(e)[:50]}{ReportStyler.ENDC}")
                         import traceback; traceback.print_exc()

                print(f"│ {'─'*138}")
                del model
                clear_memory()

    print("\nBenchmark Complete. Copy these tables to your paper appendix.")

if __name__ == "__main__":
    run_experiment()


  Q1 UNLEARNING BENCHMARK: AG NEWS


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

│ PRE-TRAINED  │ F-Acc: 0.000 │ R-Acc: 0.308 │ All-Acc: 0.231 │ ECE: 0.034 │ MIA: 0.378
│ FINE-TUNED   │ F-Acc: 0.615 │ R-Acc: 0.868 │ All-Acc: 0.805 │ ECE: 0.065 │ MIA: 0.196
Skipping: Model failed to learn target class.

┌── SCENARIO: Forget 'World' | N=100 | Seed=42 ────────────────────────────────────────────────────────────────────────────────
│ PRE-TRAINED  │ F-Acc: 0.005 │ R-Acc: 0.333 │ All-Acc: 0.251 │ ECE: 0.110 │ MIA: 0.664
│ FINE-TUNED   │ F-Acc: 0.790 │ R-Acc: 0.857 │ All-Acc: 0.840 │ ECE: 0.027 │ MIA: 0.342
│ ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
│ METHOD     │ F-ACC    │ R-ACC    │ ALL-ACC  │ ECE      │ MIA-AUC  │ PRIVACY  │ H-SCORE  │ TIME   
│ ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
│ GA+GD      │ 0.000    │ 0.853    │ 0.640    │ 0.276    │ 0.000    │ 0.000    │ 0.921    │

# New Methods

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, logging
from datasets import load_dataset
import numpy as np
import copy
import time
from collections import defaultdict
import warnings
import gc
from typing import Dict, List
from dataclasses import dataclass
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

warnings.filterwarnings('ignore')
logging.set_verbosity_error()

BEST_BASELINE_H_SCORE = 0.939
BEST_BASELINE_METHOD = "NPO"

@dataclass
class ExperimentConfig:
    MODEL_NAME: str = "distilbert-base-uncased"
    DEVICE: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BATCH_SIZE: int = 16
    MAX_LENGTH: int = 128
    LEARNING_RATE: float = 2e-5
    FINETUNE_EPOCHS: int = 3
    RETAIN_SAMPLES_PER_CLASS: int = 200
    FORGET_SCENARIOS: List[int] = None
    SEED: int = 42
    NUM_CLASSES: int = 4
    
    def __post_init__(self):
        if self.FORGET_SCENARIOS is None:
            self.FORGET_SCENARIOS = [50, 100, 200]


class ReportStyler:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

    @staticmethod
    def print_banner(text):
        print(f"\n{ReportStyler.BOLD}{'='*140}{ReportStyler.ENDC}")
        print(f"{ReportStyler.BOLD}{ReportStyler.HEADER}  {text.upper()}{ReportStyler.ENDC}")
        print(f"{ReportStyler.BOLD}{'='*140}{ReportStyler.ENDC}")

    @staticmethod
    def print_scenario_header(cls_name, n, seed):
        print(f"\n{ReportStyler.CYAN}┌── SCENARIO: Forget '{cls_name}' | N={n} | Seed={seed} {'─'*80}{ReportStyler.ENDC}")

    @staticmethod
    def print_baseline_row(name, f_acc, r_acc, all_acc, ece, mia_auc):
        print(f"│ {ReportStyler.YELLOW}{name:<12}{ReportStyler.ENDC} │ F-Acc: {f_acc:.3f} │ R-Acc: {r_acc:.3f} │ All-Acc: {all_acc:.3f} │ ECE: {ece:.3f} │ MIA: {mia_auc:.3f}")

    @staticmethod
    def print_table_header():
        print(f"│ {'─'*138}")
        print(f"│ {ReportStyler.BOLD}{'METHOD':<12} │ {'F-ACC':<8} │ {'R-ACC':<8} │ {'ALL-ACC':<8} │ {'ECE':<8} │ {'MIA-AUC':<8} │ {'PRIVACY':<8} │ {'H-SCORE':<8} │ {'TIME':<7}{ReportStyler.ENDC}")
        print(f"│ {'─'*138}")

    @staticmethod
    def print_result_row(method, f_acc, r_acc, all_acc, ece, mia, priv, h_score, elapsed):
        c_f = ReportStyler.GREEN if f_acc < 0.05 else ""
        c_r = ReportStyler.FAIL if r_acc < 0.7 else ""
        c_h = ReportStyler.BOLD + ReportStyler.GREEN if h_score > BEST_BASELINE_H_SCORE else ""
        print(f"│ {method:<12} │ {c_f}{f_acc:<8.3f}{ReportStyler.ENDC} │ {c_r}{r_acc:<8.3f}{ReportStyler.ENDC} │ {all_acc:<8.3f} │ {ece:<8.3f} │ {mia:<8.3f} │ {priv:<8.3f} │ {c_h}{h_score:<8.3f}{ReportStyler.ENDC} │ {elapsed:<7.1f}")

    @staticmethod
    def print_new_best(method, h_score, scenario):
        print(f"\n{ReportStyler.BOLD}{ReportStyler.GREEN}{'★'*60}{ReportStyler.ENDC}")
        print(f"{ReportStyler.BOLD}{ReportStyler.GREEN}  🏆 NEW SOTA! '{method}' H-Score={h_score:.4f}{ReportStyler.ENDC}")
        print(f"{ReportStyler.BOLD}{ReportStyler.GREEN}  Beats {BEST_BASELINE_METHOD} ({BEST_BASELINE_H_SCORE:.4f}) by +{(h_score - BEST_BASELINE_H_SCORE)*100:.2f}%{ReportStyler.ENDC}")
        print(f"{ReportStyler.BOLD}{ReportStyler.GREEN}  Scenario: {scenario}{ReportStyler.ENDC}")
        print(f"{ReportStyler.BOLD}{ReportStyler.GREEN}{'★'*60}{ReportStyler.ENDC}\n")


def set_seed(seed: int):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


def safe_deepcopy_model(model):
    clear_memory()
    device = next(model.parameters()).device
    model_cpu = model.to('cpu')
    model_copy = copy.deepcopy(model_cpu)
    model.to(device)
    return model_copy


class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


class Evaluator:
    def __init__(self, device, config):
        self.device = device
        self.config = config
        
    def get_full_metrics(self, model, loaders, forget_class):
        model.eval()
        model.to(self.device)
        
        all_preds, all_labels, all_probs = [], [], []
        with torch.no_grad():
            for batch in loaders['test_all']:
                out = model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device))
                probs = F.softmax(out.logits, dim=-1)
                all_preds.extend(torch.argmax(probs, dim=-1).cpu().numpy())
                all_labels.extend(batch['labels'].cpu().numpy())
                all_probs.extend(probs.cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        all_probs = np.array(all_probs)
        
        acc_per_class = {}
        for c in range(self.config.NUM_CLASSES):
            mask = all_labels == c
            acc_per_class[c] = accuracy_score(all_labels[mask], all_preds[mask]) if mask.sum() > 0 else 0.0

        confidences = np.max(all_probs, axis=1)
        predictions = np.argmax(all_probs, axis=1)
        accuracies = (predictions == all_labels)
        bins = np.linspace(0, 1, 11)
        ece = 0.0
        for i in range(10):
            mask = (confidences >= bins[i]) & (confidences < bins[i+1])
            if mask.sum() > 0:
                ece += mask.sum() / len(all_labels) * abs(accuracies[mask].mean() - confidences[mask].mean())

        crit = nn.CrossEntropyLoss(reduction='none')
        f_losses, r_losses = [], []
        with torch.no_grad():
            for batch in loaders['forget_train']:
                out = model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device))
                f_losses.extend(crit(out.logits, batch['labels'].to(self.device)).cpu().numpy())
            for batch in loaders['retain_test']:
                out = model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device))
                r_losses.extend(crit(out.logits, batch['labels'].to(self.device)).cpu().numpy())
        
        y_true = np.concatenate([np.ones(len(f_losses)), np.zeros(len(r_losses))])
        y_score = np.concatenate([-np.array(f_losses), -np.array(r_losses)])
        try:
            mia_auc = roc_auc_score(y_true, y_score)
        except:
            mia_auc = 0.5
        
        privacy_score = 1.0 - (2 * abs(mia_auc - 0.5))

        f_acc = acc_per_class[forget_class]
        r_acc = np.mean([acc_per_class[c] for c in range(4) if c != forget_class])
        
        eff = 1.0 - f_acc
        pres = r_acc
        h_score = 2 * (eff * pres) / (eff + pres + 1e-6)

        return {
            'f_acc': f_acc,
            'r_acc': r_acc,
            'all_acc': accuracy_score(all_labels, all_preds),
            'ece': ece,
            'mia_auc': mia_auc,
            'privacy_score': privacy_score,
            'h_score': h_score
        }


class BaseUnlearner:
    def __init__(self, model, device, loaders, config, forget_class):
        self.model = safe_deepcopy_model(model).to(device)
        self.device = device
        self.loaders = loaders
        self.config = config
        self.forget_class = forget_class
        self.ref_model = safe_deepcopy_model(model).to(device)
        self.ref_model.eval()
        for p in self.ref_model.parameters():
            p.requires_grad = False
    
    def get_transformer_layers(self):
        return list(self.model.distilbert.transformer.layer)
    
    def _kl_loss(self, logits1, logits2):
        return F.kl_div(
            F.log_softmax(logits1, dim=-1),
            F.softmax(logits2, dim=-1),
            reduction='batchmean'
        )


# ============================================================================
# PROVEN CHAMPIONS (Keep these)
# ============================================================================

class GradientConflictMinimization(BaseUnlearner):
    """CHAMPION - Tied SOTA at 0.933"""
    def run(self):
        retain_grad_signature = {}
        for name, param in self.model.named_parameters():
            if 'transformer.layer' in name:
                retain_grad_signature[name] = torch.zeros_like(param)
        
        self.model.train()
        for batch in list(self.loaders['retain_train'])[:10]:
            ids = batch['input_ids'].to(self.device)
            mask = batch['attention_mask'].to(self.device)
            labels = batch['labels'].to(self.device)
            
            self.model.zero_grad()
            out = self.model(ids, attention_mask=mask, labels=labels)
            out.loss.backward()
            
            for name, param in self.model.named_parameters():
                if name in retain_grad_signature and param.grad is not None:
                    retain_grad_signature[name] += torch.sign(param.grad.data)
        
        for name in retain_grad_signature:
            retain_grad_signature[name] = torch.sign(retain_grad_signature[name])
        
        opt = torch.optim.AdamW(self.model.parameters(), lr=3e-5)
        
        for epoch in range(5):  # More epochs
            for batch in self.loaders['forget_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model(ids, attention_mask=mask, labels=labels)
                (-out.loss).backward()
                
                with torch.no_grad():
                    for name, param in self.model.named_parameters():
                        if name in retain_grad_signature and param.grad is not None:
                            forget_grad_sign = torch.sign(param.grad.data)
                            conflict_mask = (forget_grad_sign == retain_grad_signature[name]).float()
                            param.grad.data *= conflict_mask
                
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            
            for batch in self.loaders['retain_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.7 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


class HybridFFNGradientConflict(BaseUnlearner):
    """STRONG - 0.931"""
    def run(self):
        layers = self.get_transformer_layers()
        
        # PHASE 1: FFN Pruning
        forget_acts = {i: [] for i in range(len(layers))}
        retain_acts = {i: [] for i in range(len(layers))}
        
        self.model.eval()
        
        for i, layer in enumerate(layers):
            with torch.no_grad():
                for batch in list(self.loaders['forget_train'])[:10]:
                    ids = batch['input_ids'].to(self.device)
                    mask = batch['attention_mask'].to(self.device)
                    
                    out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                    h = out.hidden_states[i + 1]
                    ffn_out = layer.ffn.lin1(h)
                    forget_acts[i].append(ffn_out.mean(dim=(0, 1)).cpu())
                
                for batch in list(self.loaders['retain_train'])[:10]:
                    ids = batch['input_ids'].to(self.device)
                    mask = batch['attention_mask'].to(self.device)
                    
                    out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                    h = out.hidden_states[i + 1]
                    ffn_out = layer.ffn.lin1(h)
                    retain_acts[i].append(ffn_out.mean(dim=(0, 1)).cpu())
        
        with torch.no_grad():
            for i, layer in enumerate(layers):
                if forget_acts[i] and retain_acts[i]:
                    f_act = torch.stack(forget_acts[i]).mean(dim=0).to(self.device)
                    r_act = torch.stack(retain_acts[i]).mean(dim=0).to(self.device)
                    
                    diff = (f_act - r_act)
                    threshold = diff.quantile(0.75)
                    prune_mask = (diff < threshold).float()
                    
                    layer.ffn.lin1.weight.data *= prune_mask.unsqueeze(1)
                    if layer.ffn.lin1.bias is not None:
                        layer.ffn.lin1.bias.data *= prune_mask
        
        # PHASE 2: Gradient conflict
        retain_grad_signature = {}
        for name, param in self.model.named_parameters():
            if 'transformer.layer' in name:
                retain_grad_signature[name] = torch.zeros_like(param)
        
        self.model.train()
        for batch in list(self.loaders['retain_train'])[:10]:
            ids = batch['input_ids'].to(self.device)
            mask = batch['attention_mask'].to(self.device)
            labels = batch['labels'].to(self.device)
            
            self.model.zero_grad()
            out = self.model(ids, attention_mask=mask, labels=labels)
            out.loss.backward()
            
            for name, param in self.model.named_parameters():
                if name in retain_grad_signature and param.grad is not None:
                    retain_grad_signature[name] += torch.sign(param.grad.data)
        
        for name in retain_grad_signature:
            retain_grad_signature[name] = torch.sign(retain_grad_signature[name])
        
        opt = torch.optim.AdamW(self.model.parameters(), lr=2.5e-5)
        
        for epoch in range(4):
            for batch in self.loaders['forget_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model(ids, attention_mask=mask, labels=labels)
                (-out.loss).backward()
                
                with torch.no_grad():
                    for name, param in self.model.named_parameters():
                        if name in retain_grad_signature and param.grad is not None:
                            forget_grad_sign = torch.sign(param.grad.data)
                            conflict_mask = (forget_grad_sign == retain_grad_signature[name]).float()
                            param.grad.data *= conflict_mask
                
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            
            for batch in self.loaders['retain_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.7 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


class AlternatingDirectionUnlearning(BaseUnlearner):
    """FAST - 0.922 in 4.4s"""
    def run(self):
        opt = torch.optim.AdamW(self.model.parameters(), lr=2e-5)
        self.model.train()
        
        forget_iter = iter(self.loaders['forget_train'])
        retain_iter = iter(self.loaders['retain_train'])
        
        n_steps = min(len(self.loaders['forget_train']), len(self.loaders['retain_train'])) * 5  # More steps
        
        for step in range(n_steps):
            if step % 2 == 0:
                try:
                    batch = next(forget_iter)
                except StopIteration:
                    forget_iter = iter(self.loaders['forget_train'])
                    batch = next(forget_iter)
                
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model(ids, attention_mask=mask, labels=labels)
                (-out.loss).backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            else:
                try:
                    batch = next(retain_iter)
                except StopIteration:
                    retain_iter = iter(self.loaders['retain_train'])
                    batch = next(retain_iter)
                
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.8 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
        
        return self.model


# ============================================================================
# NOVEL METHOD 1: KMeans Neuron Clustering (KNC)
# ============================================================================

class KMeansNeuronClustering(BaseUnlearner):
    """
    NOVEL: Cluster neurons by activation patterns, mask forget-dominant clusters.
    Theory: Neurons work in functional groups - cluster-level intervention.
    """
    def run(self):
        layers = self.get_transformer_layers()
        
        for i, layer in enumerate(layers):
            # Collect activation patterns for this layer
            forget_patterns = []
            retain_patterns = []
            
            self.model.eval()
            with torch.no_grad():
                # Forget patterns
                for batch in list(self.loaders['forget_train'])[:10]:
                    ids = batch['input_ids'].to(self.device)
                    mask = batch['attention_mask'].to(self.device)
                    
                    out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                    h = out.hidden_states[i + 1]
                    acts = layer.ffn.lin1(h).mean(dim=(0, 1))  # Average over batch and sequence
                    forget_patterns.append(acts.cpu().numpy())
                
                # Retain patterns
                for batch in list(self.loaders['retain_train'])[:10]:
                    ids = batch['input_ids'].to(self.device)
                    mask = batch['attention_mask'].to(self.device)
                    
                    out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                    h = out.hidden_states[i + 1]
                    acts = layer.ffn.lin1(h).mean(dim=(0, 1))
                    retain_patterns.append(acts.cpu().numpy())
            
            if not forget_patterns or not retain_patterns:
                continue
            
            # Stack: rows=samples, cols=neurons
            forget_mat = np.stack(forget_patterns)  # (n_batches, n_neurons)
            retain_mat = np.stack(retain_patterns)
            
            # Cluster neurons based on response patterns
            n_neurons = forget_mat.shape[1]
            # Transpose: rows=neurons, cols=samples
            neuron_responses = np.concatenate([forget_mat.T, retain_mat.T], axis=1)  # (n_neurons, n_samples)
            
            n_clusters = min(8, n_neurons // 200)
            if n_clusters < 2:
                n_clusters = 2
            
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
            cluster_labels = kmeans.fit_predict(neuron_responses)
            
            # For each cluster, compute forget vs retain affinity
            cluster_forget_score = np.zeros(n_clusters)
            for c in range(n_clusters):
                cluster_mask = cluster_labels == c
                if cluster_mask.sum() > 0:
                    # Average activation for neurons in this cluster
                    f_mean = forget_mat[:, cluster_mask].mean()
                    r_mean = retain_mat[:, cluster_mask].mean()
                    cluster_forget_score[c] = f_mean - r_mean
            
            # Identify forget-dominant clusters
            threshold = np.percentile(cluster_forget_score, 70)
            forget_clusters = cluster_forget_score >= threshold
            
            # Create neuron-level mask
            neuron_mask = torch.ones(n_neurons, dtype=torch.float32, device=self.device)
            for c in range(n_clusters):
                if forget_clusters[c]:
                    cluster_neurons = cluster_labels == c
                    neuron_mask[cluster_neurons] = 0.2  # Aggressive dampening
            
            # Apply mask
            with torch.no_grad():
                layer.ffn.lin1.weight.data *= neuron_mask.unsqueeze(1)
                if layer.ffn.lin1.bias is not None:
                    layer.ffn.lin1.bias.data *= neuron_mask
        
        # Repair
        opt = torch.optim.AdamW(self.model.parameters(), lr=1.5e-5)
        self.model.train()
        
        for epoch in range(4):
            for batch in self.loaders['retain_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.8 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


# ============================================================================
# NOVEL METHOD 2: PCA-Protected Gradient Ascent (PPGA)
# ============================================================================

class PCAProtectedGradientAscent(BaseUnlearner):
    """
    NOVEL: Use PCA to find retain principal components, project gradients away.
    Theory: Retain knowledge lives in a low-dimensional subspace.
    """
    def run(self):
        # Collect retain activations from middle layer
        layers = self.get_transformer_layers()
        mid_layer = len(layers) // 2
        
        retain_activations = []
        self.model.eval()
        
        with torch.no_grad():
            for batch in list(self.loaders['retain_train'])[:15]:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                
                out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                acts = out.hidden_states[mid_layer + 1][:, 0, :]  # CLS token
                retain_activations.append(acts.cpu().numpy())
        
        retain_activations = np.vstack(retain_activations)  # (n_samples, 768)
        
        # PCA to find principal components
        pca = PCA(n_components=16)  # Top 16 components
        pca.fit(retain_activations)
        principal_components = torch.tensor(pca.components_.T, dtype=torch.float32, device=self.device)  # (768, 16)
        
        # Gradient ascent with PCA projection
        opt = torch.optim.AdamW(self.model.parameters(), lr=3e-5)
        self.model.train()
        
        for epoch in range(5):
            for batch in self.loaders['forget_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model(ids, attention_mask=mask, labels=labels)
                (-out.loss).backward()
                
                # Project gradients away from retain PC space
                # Only for FFN layers
                with torch.no_grad():
                    for name, param in self.model.named_parameters():
                        if 'ffn' in name and 'weight' in name and param.grad is not None:
                            if param.grad.shape[1] == 768:  # Input from hidden states
                                # Project each row of gradient
                                grad = param.grad.data  # (out_dim, 768)
                                for pc_idx in range(principal_components.shape[1]):
                                    pc = principal_components[:, pc_idx]  # (768,)
                                    # Remove component in PC direction
                                    projection = (grad @ pc.unsqueeze(1)) @ pc.unsqueeze(0)
                                    grad -= 0.8 * projection
                                param.grad.data = grad
                
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            
            # Repair
            for batch in self.loaders['retain_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.7 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


# ============================================================================
# NOVEL METHOD 3: Retain Anchor Regularization (RAR)
# ============================================================================

class RetainAnchorRegularization(BaseUnlearner):
    """
    NOVEL: Sample retain "anchors", regularize to stay close during forgetting.
    Theory: Anchors prevent drift into regions that hurt retain performance.
    """
    def run(self):
        # Sample anchor embeddings from retain set
        layers = self.get_transformer_layers()
        n_layers = len(layers)
        
        anchors = {i: [] for i in range(n_layers)}
        
        self.model.eval()
        with torch.no_grad():
            for batch in list(self.loaders['retain_train'])[:8]:  # 8 batches = ~128 samples
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                
                out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                
                for layer_idx in range(n_layers):
                    layer_hidden = out.hidden_states[layer_idx + 1][:, 0, :]  # CLS token
                    anchors[layer_idx].append(layer_hidden.cpu())
        
        # Stack anchors
        for layer_idx in range(n_layers):
            if anchors[layer_idx]:
                anchors[layer_idx] = torch.cat(anchors[layer_idx], dim=0).to(self.device)  # (n_anchors, 768)
        
        # Alternating training with anchor regularization
        opt = torch.optim.AdamW(self.model.parameters(), lr=2e-5)
        self.model.train()
        
        forget_iter = iter(self.loaders['forget_train'])
        retain_iter = iter(self.loaders['retain_train'])
        
        n_steps = min(len(self.loaders['forget_train']), len(self.loaders['retain_train'])) * 5
        
        for step in range(n_steps):
            if step % 2 == 0:
                # Forget with anchor regularization
                try:
                    batch = next(forget_iter)
                except StopIteration:
                    forget_iter = iter(self.loaders['forget_train'])
                    batch = next(forget_iter)
                
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                
                # Forget loss
                logits = self.model.classifier(out.last_hidden_state[:, 0, :])
                forget_loss = F.cross_entropy(logits, labels)
                
                # Anchor regularization: minimize drift from anchors
                anchor_loss = 0
                for layer_idx in [len(layers) // 2, len(layers) * 2 // 3]:  # Middle and upper layers
                    if layer_idx in anchors and len(anchors[layer_idx]) > 0:
                        current_hidden = out.hidden_states[layer_idx + 1][:, 0, :]  # (batch, 768)
                        anchor_samples = anchors[layer_idx][:min(32, len(anchors[layer_idx]))]  # Sample 32 anchors
                        
                        # Distance to nearest anchor
                        distances = torch.cdist(current_hidden, anchor_samples)  # (batch, n_anchors)
                        min_distances = distances.min(dim=1)[0]  # (batch,)
                        anchor_loss += min_distances.mean()
                
                total_loss = -forget_loss + 0.3 * anchor_loss  # Forget but stay close to anchors
                
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            else:
                # Retain step
                try:
                    batch = next(retain_iter)
                except StopIteration:
                    retain_iter = iter(self.loaders['retain_train'])
                    batch = next(retain_iter)
                
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.8 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


# ============================================================================
# NOVEL METHOD 4: Critical Path Preservation (CPP)
# ============================================================================

class CriticalPathPreservation(BaseUnlearner):
    """
    NOVEL: Identify critical parameters for retain (high gradient magnitude), freeze them.
    Theory: Some parameters are more important - protect them during forgetting.
    """
    def run(self):
        # Identify critical parameters for retain
        param_importance = {}
        for name, param in self.model.named_parameters():
            if 'transformer.layer' in name:
                param_importance[name] = torch.zeros_like(param)
        
        self.model.train()
        for batch in list(self.loaders['retain_train'])[:10]:
            ids = batch['input_ids'].to(self.device)
            mask = batch['attention_mask'].to(self.device)
            labels = batch['labels'].to(self.device)
            
            self.model.zero_grad()
            out = self.model(ids, attention_mask=mask, labels=labels)
            out.loss.backward()
            
            for name, param in self.model.named_parameters():
                if name in param_importance and param.grad is not None:
                    param_importance[name] += param.grad.data.abs()
        
        # Normalize importance
        for name in param_importance:
            param_importance[name] = param_importance[name] / (param_importance[name].max() + 1e-8)
        
        # Gradient ascent with parameter-wise learning rates
        opt = torch.optim.AdamW(self.model.parameters(), lr=2e-5)
        
        for epoch in range(5):
            for batch in self.loaders['forget_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model(ids, attention_mask=mask, labels=labels)
                (-out.loss).backward()
                
                # Scale gradients inversely by importance (protect important params)
                with torch.no_grad():
                    for name, param in self.model.named_parameters():
                        if name in param_importance and param.grad is not None:
                            protection = param_importance[name]
                            param.grad.data *= (1.0 - 0.8 * protection)  # High importance = less update
                
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            
            # Repair
            for batch in self.loaders['retain_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.7 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


# ============================================================================
# NOVEL METHOD 5: Adversarial Contrastive Unlearning (ACU)
# ============================================================================

class AdversarialContrastiveUnlearning(BaseUnlearner):
    """
    NOVEL: Push forget embeddings toward retain distribution via contrastive loss.
    Theory: Make forget samples indistinguishable from retain in embedding space.
    """
    def run(self):
        layers = self.get_transformer_layers()
        target_layer = len(layers) * 2 // 3
        
        # Collect retain embedding centers
        retain_embeds = []
        self.model.eval()
        
        with torch.no_grad():
            for batch in list(self.loaders['retain_train'])[:12]:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                
                out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                embeds = out.hidden_states[target_layer + 1][:, 0, :]
                retain_embeds.append(embeds.cpu())
        
        retain_embeds = torch.cat(retain_embeds, dim=0).to(self.device)
        retain_center = retain_embeds.mean(dim=0)
        
        # Training
        opt = torch.optim.AdamW(self.model.parameters(), lr=3e-5)
        self.model.train()
        
        for epoch in range(5):
            # Push forget toward retain
            for batch in self.loaders['forget_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                
                out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                forget_embeds = out.hidden_states[target_layer + 1][:, 0, :]
                
                # Contrastive: attract to retain center
                attract_loss = F.mse_loss(forget_embeds, retain_center.unsqueeze(0).expand_as(forget_embeds))
                
                # Also do gradient ascent on classification
                logits = self.model.classifier(out.last_hidden_state[:, 0, :])
                labels = batch['labels'].to(self.device)
                class_loss = F.cross_entropy(logits, labels)
                
                total_loss = -0.7 * class_loss + 0.3 * attract_loss
                
                opt.zero_grad()
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            
            # Repair
            for batch in self.loaders['retain_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.8 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


# ============================================================================
# NOVEL METHOD 6: Hierarchical Layer Unlearning (HLU)
# ============================================================================

class HierarchicalLayerUnlearning(BaseUnlearner):
    """
    NOVEL: Different forgetting rates per layer (aggressive in upper, gentle in lower).
    Theory: Lower layers = general features, upper layers = class-specific.
    """
    def run(self):
        layers = self.get_transformer_layers()
        n_layers = len(layers)
        
        # Define layer-wise forgetting strength (0=no forget, 1=max forget)
        layer_strength = torch.linspace(0.3, 1.0, n_layers)  # Increase from bottom to top
        
        # Alternating training with layer-wise LR
        forget_iter = iter(self.loaders['forget_train'])
        retain_iter = iter(self.loaders['retain_train'])
        
        n_steps = min(len(self.loaders['forget_train']), len(self.loaders['retain_train'])) * 5
        
        self.model.train()
        
        for step in range(n_steps):
            if step % 2 == 0:
                # Forget step with layer-wise LR
                try:
                    batch = next(forget_iter)
                except StopIteration:
                    forget_iter = iter(self.loaders['forget_train'])
                    batch = next(forget_iter)
                
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model(ids, attention_mask=mask, labels=labels)
                (-out.loss).backward()
                
                # Layer-wise gradient scaling
                with torch.no_grad():
                    for layer_idx, layer in enumerate(layers):
                        strength = layer_strength[layer_idx].item()
                        for param in layer.parameters():
                            if param.grad is not None:
                                param.grad.data *= strength
                
                base_lr = 2e-5
                with torch.no_grad():
                    for param in self.model.parameters():
                        if param.grad is not None:
                            param.data += base_lr * param.grad
            else:
                # Retain step
                try:
                    batch = next(retain_iter)
                except StopIteration:
                    retain_iter = iter(self.loaders['retain_train'])
                    batch = next(retain_iter)
                
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.8 * kl_loss
                
                opt = torch.optim.AdamW(self.model.parameters(), lr=1.5e-5)
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


# ============================================================================
# Data Loading and Experiment Runner
# ============================================================================

def load_data(config, tokenizer):
    raw = load_dataset("ag_news")
    data = defaultdict(lambda: defaultdict(list))
    for split in ['train', 'test']:
        for item in raw[split]:
            data[split][item['label']].append(item['text'])
    return data


def make_loader(texts, labels, tokenizer, batch_size, shuffle=False):
    ds = TextClassificationDataset(texts, labels, tokenizer, 128)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)


def run_experiment():
    conf = ExperimentConfig()
    ReportStyler.print_banner("Transformer Unlearning v8.0 - SOTA Breakthrough 🎯")
    
    print(f"\n{ReportStyler.BOLD}Configuration:{ReportStyler.ENDC}")
    print(f"  Device: {conf.DEVICE}")
    print(f"  Model: {conf.MODEL_NAME}")
    print(f"  Seed: {conf.SEED}")
    print(f"  Forget Scenarios: {conf.FORGET_SCENARIOS}")
    print(f"\n{ReportStyler.BOLD}Current SOTA:{ReportStyler.ENDC}")
    print(f"  Best H-Score: {BEST_BASELINE_H_SCORE:.4f} ({BEST_BASELINE_METHOD})")
    print(f"\n{ReportStyler.BOLD}Method Portfolio (10 methods):{ReportStyler.ENDC}")
    print(f"  PROVEN (3): GCM, HFGCM, ADU")
    print(f"  NOVEL-CLUSTERING (1): KNC (KMeans neuron clustering)")
    print(f"  NOVEL-SUBSPACE (2): PPGA (PCA protection), RAR (anchor regularization)")
    print(f"  NOVEL-PATH (1): CPP (critical path preservation)")
    print(f"  NOVEL-CONTRASTIVE (1): ACU (adversarial contrastive)")
    print(f"  NOVEL-HIERARCHICAL (1): HLU (layer-wise forgetting)")
    
    set_seed(conf.SEED)
    
    tokenizer = AutoTokenizer.from_pretrained(conf.MODEL_NAME)
    raw_data = load_data(conf, tokenizer)
    evaluator = Evaluator(conf.DEVICE, conf)

    final_results = []
    best_new_score = 0.0
    best_new_method = None
    best_new_scenario = None

    for f_cls in range(conf.NUM_CLASSES):
        class_names = ["World", "Sports", "Business", "Sci/Tech"]
        target_name = class_names[f_cls]
        
        for n_forget in conf.FORGET_SCENARIOS:
            ReportStyler.print_scenario_header(target_name, n_forget, conf.SEED)
            
            f_train_txt = raw_data['train'][f_cls][:n_forget]
            f_train_lbl = [f_cls] * n_forget
            
            r_train_txt, r_train_lbl = [], []
            for c in range(4):
                if c != f_cls:
                    r_train_txt.extend(raw_data['train'][c][:conf.RETAIN_SAMPLES_PER_CLASS])
                    r_train_lbl.extend([c] * conf.RETAIN_SAMPLES_PER_CLASS)
            
            t_f_txt = raw_data['test'][f_cls][:200]
            t_f_lbl = [f_cls] * 200
            t_r_txt, t_r_lbl = [], []
            for c in range(4):
                if c != f_cls:
                    t_r_txt.extend(raw_data['test'][c][:200])
                    t_r_lbl.extend([c] * 200)

            loaders = {
                'forget_train': make_loader(f_train_txt, f_train_lbl, tokenizer, conf.BATCH_SIZE, True),
                'retain_train': make_loader(r_train_txt, r_train_lbl, tokenizer, conf.BATCH_SIZE, True),
                'forget_test': make_loader(t_f_txt, t_f_lbl, tokenizer, conf.BATCH_SIZE, False),
                'retain_test': make_loader(t_r_txt, t_r_lbl, tokenizer, conf.BATCH_SIZE, False),
                'test_all': make_loader(t_f_txt + t_r_txt, t_f_lbl + t_r_lbl, tokenizer, conf.BATCH_SIZE, False)
            }

            model = AutoModelForSequenceClassification.from_pretrained(conf.MODEL_NAME, num_labels=4).to(conf.DEVICE)
            base_metrics = evaluator.get_full_metrics(model, loaders, f_cls)
            ReportStyler.print_baseline_row("PRE-TRAINED", base_metrics['f_acc'], base_metrics['r_acc'], base_metrics['all_acc'], base_metrics['ece'], base_metrics['mia_auc'])

            model.train()
            opt = torch.optim.AdamW(model.parameters(), lr=conf.LEARNING_RATE)
            combined = ConcatDataset([loaders['forget_train'].dataset, loaders['retain_train'].dataset])
            train_l = DataLoader(combined, batch_size=conf.BATCH_SIZE, shuffle=True)
            
            for _ in range(conf.FINETUNE_EPOCHS):
                for batch in train_l:
                    out = model(batch['input_ids'].to(conf.DEVICE), attention_mask=batch['attention_mask'].to(conf.DEVICE), labels=batch['labels'].to(conf.DEVICE))
                    opt.zero_grad()
                    out.loss.backward()
                    opt.step()
            
            ft_metrics = evaluator.get_full_metrics(model, loaders, f_cls)
            ReportStyler.print_baseline_row("FINE-TUNED", ft_metrics['f_acc'], ft_metrics['r_acc'], ft_metrics['all_acc'], ft_metrics['ece'], ft_metrics['mia_auc'])
            
            if ft_metrics['f_acc'] < 0.7:
                print(f"{ReportStyler.FAIL}│ Skipping: Model failed to learn target class (F-Acc < 0.7).{ReportStyler.ENDC}")
                del model
                clear_memory()
                continue

            ReportStyler.print_table_header()
            
            methods = [
                # Proven champions
                ("GCM", GradientConflictMinimization),
                ("HFGCM", HybridFFNGradientConflict),
                ("ADU", AlternatingDirectionUnlearning),
                
                # Novel methods
                ("KNC", KMeansNeuronClustering),
                ("PPGA", PCAProtectedGradientAscent),
                ("RAR", RetainAnchorRegularization),
                ("CPP", CriticalPathPreservation),
                ("ACU", AdversarialContrastiveUnlearning),
                ("HLU", HierarchicalLayerUnlearning),
            ]

            ft_model_cpu = safe_deepcopy_model(model).to('cpu')

            for name, Cls in methods:
                try:
                    current_model = safe_deepcopy_model(ft_model_cpu).to(conf.DEVICE)
                    
                    t0 = time.time()
                    unlearner = Cls(current_model, conf.DEVICE, loaders, conf, f_cls)
                    unlearned_model = unlearner.run()
                    dur = time.time() - t0
                    
                    res = evaluator.get_full_metrics(unlearned_model, loaders, f_cls)
                    
                    ReportStyler.print_result_row(
                        name, res['f_acc'], res['r_acc'], res['all_acc'],
                        res['ece'], res['mia_auc'], res['privacy_score'],
                        res['h_score'], dur
                    )
                    
                    scenario_str = f"{target_name}/N={n_forget}/Seed={conf.SEED}"
                    
                    if res['h_score'] > BEST_BASELINE_H_SCORE:
                        ReportStyler.print_new_best(name, res['h_score'], scenario_str)
                    
                    if res['h_score'] > best_new_score:
                        best_new_score = res['h_score']
                        best_new_method = name
                        best_new_scenario = scenario_str
                    
                    final_results.append({
                        'class': target_name, 'n': n_forget, 'method': name,
                        **res, 'time': dur
                    })
                    
                    del unlearned_model, unlearner, current_model
                    clear_memory()
                    
                except Exception as e:
                    print(f"│ {ReportStyler.FAIL}{name:<12} │ ERROR: {str(e)[:60]}{ReportStyler.ENDC}")
                    import traceback
                    traceback.print_exc()

            print(f"│ {'─'*138}")
            del model, ft_model_cpu
            clear_memory()

    print("\n" + "=" * 140)
    print(f"{ReportStyler.BOLD}🏆 FINAL RESULTS - PUBLICATION READY 🏆{ReportStyler.ENDC}")
    print("=" * 140)
    
    if final_results:
        method_stats = defaultdict(lambda: {'f_acc': [], 'r_acc': [], 'h_score': [], 'privacy': [], 'time': []})
        
        for r in final_results:
            m = r['method']
            method_stats[m]['f_acc'].append(r['f_acc'])
            method_stats[m]['r_acc'].append(r['r_acc'])
            method_stats[m]['h_score'].append(r['h_score'])
            method_stats[m]['privacy'].append(r['privacy_score'])
            method_stats[m]['time'].append(r['time'])
        
        print(f"\n{'Method':<12} │ {'F-Acc':<16} │ {'R-Acc':<16} │ {'H-Score':<16} │ {'Privacy':<16} │ {'Time':<10}")
        print("-" * 105)
        
        ranking = []
        for method, stats in sorted(method_stats.items()):
            f_mean, f_std = np.mean(stats['f_acc']), np.std(stats['f_acc'])
            r_mean, r_std = np.mean(stats['r_acc']), np.std(stats['r_acc'])
            h_mean, h_std = np.mean(stats['h_score']), np.std(stats['h_score'])
            p_mean, p_std = np.mean(stats['privacy']), np.std(stats['privacy'])
            t_mean = np.mean(stats['time'])
            
            beat_baseline = "🏆" if h_mean > BEST_BASELINE_H_SCORE else ("=" if abs(h_mean - BEST_BASELINE_H_SCORE) < 0.001 else "")
            print(f"{method:<12} │ {f_mean:.3f}±{f_std:.3f}      │ {r_mean:.3f}±{r_std:.3f}      │ {h_mean:.3f}±{h_std:.3f} {beat_baseline:<3} │ {p_mean:.3f}±{p_std:.3f}      │ {t_mean:.1f}s")
            ranking.append((method, h_mean, f_mean, r_mean, p_mean, t_mean))
        
        ranking.sort(key=lambda x: (-x[1], x[5]))
        
        print("\n" + "=" * 140)
        print(f"{ReportStyler.BOLD}🎯 FINAL RANKING 🎯{ReportStyler.ENDC}")
        print("=" * 140)
        for rank, (method, h, f, r, p, t) in enumerate(ranking, 1):
            if h > BEST_BASELINE_H_SCORE:
                indicator = f" {ReportStyler.GREEN}★ NEW SOTA ★{ReportStyler.ENDC}"
            elif abs(h - BEST_BASELINE_H_SCORE) < 0.001:
                indicator = f" {ReportStyler.YELLOW}= TIED ={ReportStyler.ENDC}"
            else:
                indicator = ""
            print(f"  {rank:2d}. {method:<12} H={h:.4f} F={f:.3f} R={r:.3f} P={p:.3f} T={t:.1f}s{indicator}")
        
        print("\n" + "=" * 140)
        print(f"{ReportStyler.BOLD}📊 PAPER SUMMARY 📊{ReportStyler.ENDC}")
        print("=" * 140)
        
        if best_new_score > BEST_BASELINE_H_SCORE:
            print(f"{ReportStyler.BOLD}{ReportStyler.GREEN}✨ NEW STATE-OF-THE-ART ACHIEVED! ✨{ReportStyler.ENDC}")
            print(f"   Method: {best_new_method}")
            print(f"   H-Score: {best_new_score:.4f}")
            print(f"   Improvement: +{(best_new_score - BEST_BASELINE_H_SCORE)*100:.2f}%")
            print(f"   Scenario: {best_new_scenario}")
        else:
            print(f"{ReportStyler.YELLOW}Best Method: {best_new_method} (H={best_new_score:.4f}){ReportStyler.ENDC}")
            if abs(best_new_score - BEST_BASELINE_H_SCORE) < 0.001:
                print(f"   Status: TIED with baseline")
            else:
                print(f"   Gap: {(BEST_BASELINE_H_SCORE - best_new_score)*100:.2f}%")
        
        print("\n" + "=" * 140)
        print(f"{ReportStyler.BOLD}📝 THEORETICAL CONTRIBUTIONS:{ReportStyler.ENDC}")
        print("=" * 140)
        print("  1. GCM: Gradient conflict detection separates shared vs class-specific parameters")
        print("  2. KNC: Functional neuron clustering enables cluster-level intervention")
        print("  3. PPGA: PCA subspace protection preserves retain knowledge geometry")
        print("  4. RAR: Anchor regularization prevents drift in representation space")
        print("  5. CPP: Critical path analysis identifies and protects important parameters")
        print("  6. ACU: Adversarial contrastive learning confuses class boundaries")
        print("  7. HLU: Hierarchical layer treatment exploits abstraction levels")
        print("=" * 140)

    return final_results


if __name__ == "__main__":
    results = run_experiment()


  TRANSFORMER UNLEARNING V8.0 - SOTA BREAKTHROUGH 🎯

Configuration:
  Device: cuda
  Model: distilbert-base-uncased
  Seed: 42
  Forget Scenarios: [50, 100, 200]

Current SOTA:
  Best H-Score: 0.9390 (NPO)

Method Portfolio (10 methods):
  PROVEN (3): GCM, HFGCM, ADU
  NOVEL-CLUSTERING (1): KNC (KMeans neuron clustering)
  NOVEL-SUBSPACE (2): PPGA (PCA protection), RAR (anchor regularization)
  NOVEL-PATH (1): CPP (critical path preservation)
  NOVEL-CONTRASTIVE (1): ACU (adversarial contrastive)
  NOVEL-HIERARCHICAL (1): HLU (layer-wise forgetting)

┌── SCENARIO: Forget 'World' | N=50 | Seed=42 ────────────────────────────────────────────────────────────────────────────────
│ PRE-TRAINED  │ F-Acc: 0.000 │ R-Acc: 0.308 │ All-Acc: 0.231 │ ECE: 0.034 │ MIA: 0.378
│ FINE-TUNED   │ F-Acc: 0.615 │ R-Acc: 0.868 │ All-Acc: 0.805 │ ECE: 0.065 │ MIA: 0.196
│ Skipping: Model failed to learn target class (F-Acc < 0.7).

┌── SCENARIO: Forget 'World' | N=100 | Seed=42 ─────────────────────────────

# top method deep testing 

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, logging
from datasets import load_dataset
import numpy as np
import copy
import time
from collections import defaultdict
import warnings
import gc
from typing import Dict, List
from dataclasses import dataclass
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.decomposition import PCA

warnings.filterwarnings('ignore')
logging.set_verbosity_error()

@dataclass
class ExperimentConfig:
    MODEL_NAME: str = "distilbert-base-uncased"
    DEVICE: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BATCH_SIZE: int = 16
    MAX_LENGTH: int = 128
    LEARNING_RATE: float = 2e-5
    FINETUNE_EPOCHS: int = 3
    RETAIN_SAMPLES_PER_CLASS: int = 200
    FORGET_SCENARIOS: List[int] = None
    SEEDS: List[int] = None
    NUM_CLASSES: int = 4
    
    def __post_init__(self):
        if self.SEEDS is None:
            self.SEEDS = [42, 123, 456]
        if self.FORGET_SCENARIOS is None:
            self.FORGET_SCENARIOS = [50, 100, 200]


class ReportStyler:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

    @staticmethod
    def print_banner(text):
        print(f"\n{ReportStyler.BOLD}{'='*140}{ReportStyler.ENDC}")
        print(f"{ReportStyler.BOLD}{ReportStyler.HEADER}  {text.upper()}{ReportStyler.ENDC}")
        print(f"{ReportStyler.BOLD}{'='*140}{ReportStyler.ENDC}")

    @staticmethod
    def print_scenario_header(cls_name, n, seed):
        print(f"\n{ReportStyler.CYAN}┌── SCENARIO: Forget '{cls_name}' | N={n} | Seed={seed} {'─'*80}{ReportStyler.ENDC}")

    @staticmethod
    def print_baseline_row(name, f_acc, r_acc, all_acc, ece, mia_auc):
        print(f"│ {ReportStyler.YELLOW}{name:<12}{ReportStyler.ENDC} │ F-Acc: {f_acc:.3f} │ R-Acc: {r_acc:.3f} │ All-Acc: {all_acc:.3f} │ ECE: {ece:.3f} │ MIA: {mia_auc:.3f}")

    @staticmethod
    def print_table_header():
        print(f"│ {'─'*138}")
        print(f"│ {ReportStyler.BOLD}{'METHOD':<10} │ {'F-ACC':<8} │ {'R-ACC':<8} │ {'ALL-ACC':<8} │ {'ECE':<8} │ {'MIA-AUC':<8} │ {'PRIVACY':<8} │ {'H-SCORE':<8} │ {'TIME':<7}{ReportStyler.ENDC}")
        print(f"│ {'─'*138}")

    @staticmethod
    def print_result_row(method, f_acc, r_acc, all_acc, ece, mia, priv, h_score, elapsed):
        c_f = ReportStyler.GREEN if f_acc < 0.05 else ""
        c_r = ReportStyler.FAIL if r_acc < 0.7 else ""
        c_h = ReportStyler.BOLD if h_score > 0.85 else ""
        
        print(f"│ {method:<10} │ {c_f}{f_acc:<8.3f}{ReportStyler.ENDC} │ {c_r}{r_acc:<8.3f}{ReportStyler.ENDC} │ {all_acc:<8.3f} │ {ece:<8.3f} │ {mia:<8.3f} │ {priv:<8.3f} │ {c_h}{h_score:<8.3f}{ReportStyler.ENDC} │ {elapsed:<7.1f}")


def set_seed(seed: int):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


def safe_deepcopy_model(model):
    clear_memory()
    device = next(model.parameters()).device
    model_cpu = model.to('cpu')
    model_copy = copy.deepcopy(model_cpu)
    model.to(device)
    return model_copy


class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


class Evaluator:
    def __init__(self, device, config):
        self.device = device
        self.config = config
        
    def get_full_metrics(self, model, loaders, forget_class):
        model.eval()
        model.to(self.device)
        
        all_preds, all_labels, all_probs = [], [], []
        with torch.no_grad():
            for batch in loaders['test_all']:
                out = model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device))
                probs = F.softmax(out.logits, dim=-1)
                all_preds.extend(torch.argmax(probs, dim=-1).cpu().numpy())
                all_labels.extend(batch['labels'].cpu().numpy())
                all_probs.extend(probs.cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        all_probs = np.array(all_probs)
        
        acc_per_class = {}
        for c in range(self.config.NUM_CLASSES):
            mask = all_labels == c
            acc_per_class[c] = accuracy_score(all_labels[mask], all_preds[mask]) if mask.sum() > 0 else 0.0

        confidences = np.max(all_probs, axis=1)
        predictions = np.argmax(all_probs, axis=1)
        accuracies = (predictions == all_labels)
        bins = np.linspace(0, 1, 11)
        ece = 0.0
        for i in range(10):
            mask = (confidences >= bins[i]) & (confidences < bins[i+1])
            if mask.sum() > 0:
                ece += mask.sum() / len(all_labels) * abs(accuracies[mask].mean() - confidences[mask].mean())

        crit = nn.CrossEntropyLoss(reduction='none')
        f_losses, r_losses = [], []
        with torch.no_grad():
            for batch in loaders['forget_train']:
                out = model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device))
                f_losses.extend(crit(out.logits, batch['labels'].to(self.device)).cpu().numpy())
            for batch in loaders['retain_test']:
                out = model(batch['input_ids'].to(self.device), attention_mask=batch['attention_mask'].to(self.device))
                r_losses.extend(crit(out.logits, batch['labels'].to(self.device)).cpu().numpy())
        
        y_true = np.concatenate([np.ones(len(f_losses)), np.zeros(len(r_losses))])
        y_score = np.concatenate([-np.array(f_losses), -np.array(r_losses)])
        try:
            mia_auc = roc_auc_score(y_true, y_score)
        except:
            mia_auc = 0.5
        
        privacy_score = 1.0 - (2 * abs(mia_auc - 0.5))

        f_acc = acc_per_class[forget_class]
        r_acc = np.mean([acc_per_class[c] for c in range(4) if c != forget_class])
        
        eff = 1.0 - f_acc
        pres = r_acc
        h_score = 2 * (eff * pres) / (eff + pres + 1e-6)

        return {
            'f_acc': f_acc,
            'r_acc': r_acc,
            'all_acc': accuracy_score(all_labels, all_preds),
            'ece': ece,
            'mia_auc': mia_auc,
            'privacy_score': privacy_score,
            'h_score': h_score
        }


class BaseUnlearner:
    def __init__(self, model, device, loaders, config, forget_class):
        self.model = safe_deepcopy_model(model).to(device)
        self.device = device
        self.loaders = loaders
        self.config = config
        self.forget_class = forget_class
        self.ref_model = safe_deepcopy_model(model).to(device)
        self.ref_model.eval()
        for p in self.ref_model.parameters():
            p.requires_grad = False
    
    def get_transformer_layers(self):
        return list(self.model.distilbert.transformer.layer)
    
    def _kl_loss(self, logits1, logits2):
        return F.kl_div(
            F.log_softmax(logits1, dim=-1),
            F.softmax(logits2, dim=-1),
            reduction='batchmean'
        )


# ============================================================================
# TOP 5 SOTA METHODS
# ============================================================================

class GradientConflictMinimization(BaseUnlearner):
    """#1 - H-Score: 0.9574"""
    def run(self):
        retain_grad_signature = {}
        for name, param in self.model.named_parameters():
            if 'transformer.layer' in name:
                retain_grad_signature[name] = torch.zeros_like(param)
        
        self.model.train()
        for batch in list(self.loaders['retain_train'])[:10]:
            ids = batch['input_ids'].to(self.device)
            mask = batch['attention_mask'].to(self.device)
            labels = batch['labels'].to(self.device)
            
            self.model.zero_grad()
            out = self.model(ids, attention_mask=mask, labels=labels)
            out.loss.backward()
            
            for name, param in self.model.named_parameters():
                if name in retain_grad_signature and param.grad is not None:
                    retain_grad_signature[name] += torch.sign(param.grad.data)
        
        for name in retain_grad_signature:
            retain_grad_signature[name] = torch.sign(retain_grad_signature[name])
        
        opt = torch.optim.AdamW(self.model.parameters(), lr=3e-5)
        
        for epoch in range(5):
            for batch in self.loaders['forget_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model(ids, attention_mask=mask, labels=labels)
                (-out.loss).backward()
                
                with torch.no_grad():
                    for name, param in self.model.named_parameters():
                        if name in retain_grad_signature and param.grad is not None:
                            forget_grad_sign = torch.sign(param.grad.data)
                            conflict_mask = (forget_grad_sign == retain_grad_signature[name]).float()
                            param.grad.data *= conflict_mask
                
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            
            for batch in self.loaders['retain_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.7 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


class PCAProtectedGradientAscent(BaseUnlearner):
    """#2 - H-Score: 0.9565"""
    def run(self):
        layers = self.get_transformer_layers()
        mid_layer = len(layers) // 2
        
        retain_activations = []
        self.model.eval()
        
        with torch.no_grad():
            for batch in list(self.loaders['retain_train'])[:15]:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                
                out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                acts = out.hidden_states[mid_layer + 1][:, 0, :]
                retain_activations.append(acts.cpu().numpy())
        
        retain_activations = np.vstack(retain_activations)
        
        pca = PCA(n_components=16)
        pca.fit(retain_activations)
        principal_components = torch.tensor(pca.components_.T, dtype=torch.float32, device=self.device)
        
        opt = torch.optim.AdamW(self.model.parameters(), lr=3e-5)
        self.model.train()
        
        for epoch in range(5):
            for batch in self.loaders['forget_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model(ids, attention_mask=mask, labels=labels)
                (-out.loss).backward()
                
                with torch.no_grad():
                    for name, param in self.model.named_parameters():
                        if 'ffn' in name and 'weight' in name and param.grad is not None:
                            if param.grad.shape[1] == 768:
                                grad = param.grad.data
                                for pc_idx in range(principal_components.shape[1]):
                                    pc = principal_components[:, pc_idx]
                                    projection = (grad @ pc.unsqueeze(1)) @ pc.unsqueeze(0)
                                    grad -= 0.8 * projection
                                param.grad.data = grad
                
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            
            for batch in self.loaders['retain_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.7 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


class AlternatingDirectionUnlearning(BaseUnlearner):
    """#3 - H-Score: 0.9547"""
    def run(self):
        opt = torch.optim.AdamW(self.model.parameters(), lr=2e-5)
        self.model.train()
        
        forget_iter = iter(self.loaders['forget_train'])
        retain_iter = iter(self.loaders['retain_train'])
        
        n_steps = min(len(self.loaders['forget_train']), len(self.loaders['retain_train'])) * 5
        
        for step in range(n_steps):
            if step % 2 == 0:
                try:
                    batch = next(forget_iter)
                except StopIteration:
                    forget_iter = iter(self.loaders['forget_train'])
                    batch = next(forget_iter)
                
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model(ids, attention_mask=mask, labels=labels)
                (-out.loss).backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            else:
                try:
                    batch = next(retain_iter)
                except StopIteration:
                    retain_iter = iter(self.loaders['retain_train'])
                    batch = next(retain_iter)
                
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.8 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
        
        return self.model


class CriticalPathPreservation(BaseUnlearner):
    """#4 - H-Score: 0.9538"""
    def run(self):
        param_importance = {}
        for name, param in self.model.named_parameters():
            if 'transformer.layer' in name:
                param_importance[name] = torch.zeros_like(param)
        
        self.model.train()
        for batch in list(self.loaders['retain_train'])[:10]:
            ids = batch['input_ids'].to(self.device)
            mask = batch['attention_mask'].to(self.device)
            labels = batch['labels'].to(self.device)
            
            self.model.zero_grad()
            out = self.model(ids, attention_mask=mask, labels=labels)
            out.loss.backward()
            
            for name, param in self.model.named_parameters():
                if name in param_importance and param.grad is not None:
                    param_importance[name] += param.grad.data.abs()
        
        for name in param_importance:
            param_importance[name] = param_importance[name] / (param_importance[name].max() + 1e-8)
        
        opt = torch.optim.AdamW(self.model.parameters(), lr=2e-5)
        
        for epoch in range(5):
            for batch in self.loaders['forget_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model(ids, attention_mask=mask, labels=labels)
                (-out.loss).backward()
                
                with torch.no_grad():
                    for name, param in self.model.named_parameters():
                        if name in param_importance and param.grad is not None:
                            protection = param_importance[name]
                            param.grad.data *= (1.0 - 0.8 * protection)
                
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            
            for batch in self.loaders['retain_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.7 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


class HybridFFNGradientConflict(BaseUnlearner):
    """#5 - H-Score: 0.9520"""
    def run(self):
        layers = self.get_transformer_layers()
        
        forget_acts = {i: [] for i in range(len(layers))}
        retain_acts = {i: [] for i in range(len(layers))}
        
        self.model.eval()
        
        for i, layer in enumerate(layers):
            with torch.no_grad():
                for batch in list(self.loaders['forget_train'])[:10]:
                    ids = batch['input_ids'].to(self.device)
                    mask = batch['attention_mask'].to(self.device)
                    
                    out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                    h = out.hidden_states[i + 1]
                    ffn_out = layer.ffn.lin1(h)
                    forget_acts[i].append(ffn_out.mean(dim=(0, 1)).cpu())
                
                for batch in list(self.loaders['retain_train'])[:10]:
                    ids = batch['input_ids'].to(self.device)
                    mask = batch['attention_mask'].to(self.device)
                    
                    out = self.model.distilbert(ids, attention_mask=mask, output_hidden_states=True)
                    h = out.hidden_states[i + 1]
                    ffn_out = layer.ffn.lin1(h)
                    retain_acts[i].append(ffn_out.mean(dim=(0, 1)).cpu())
        
        with torch.no_grad():
            for i, layer in enumerate(layers):
                if forget_acts[i] and retain_acts[i]:
                    f_act = torch.stack(forget_acts[i]).mean(dim=0).to(self.device)
                    r_act = torch.stack(retain_acts[i]).mean(dim=0).to(self.device)
                    
                    diff = (f_act - r_act)
                    threshold = diff.quantile(0.75)
                    prune_mask = (diff < threshold).float()
                    
                    layer.ffn.lin1.weight.data *= prune_mask.unsqueeze(1)
                    if layer.ffn.lin1.bias is not None:
                        layer.ffn.lin1.bias.data *= prune_mask
        
        retain_grad_signature = {}
        for name, param in self.model.named_parameters():
            if 'transformer.layer' in name:
                retain_grad_signature[name] = torch.zeros_like(param)
        
        self.model.train()
        for batch in list(self.loaders['retain_train'])[:10]:
            ids = batch['input_ids'].to(self.device)
            mask = batch['attention_mask'].to(self.device)
            labels = batch['labels'].to(self.device)
            
            self.model.zero_grad()
            out = self.model(ids, attention_mask=mask, labels=labels)
            out.loss.backward()
            
            for name, param in self.model.named_parameters():
                if name in retain_grad_signature and param.grad is not None:
                    retain_grad_signature[name] += torch.sign(param.grad.data)
        
        for name in retain_grad_signature:
            retain_grad_signature[name] = torch.sign(retain_grad_signature[name])
        
        opt = torch.optim.AdamW(self.model.parameters(), lr=2.5e-5)
        
        for epoch in range(4):
            for batch in self.loaders['forget_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                self.model.zero_grad()
                out = self.model(ids, attention_mask=mask, labels=labels)
                (-out.loss).backward()
                
                with torch.no_grad():
                    for name, param in self.model.named_parameters():
                        if name in retain_grad_signature and param.grad is not None:
                            forget_grad_sign = torch.sign(param.grad.data)
                            conflict_mask = (forget_grad_sign == retain_grad_signature[name]).float()
                            param.grad.data *= conflict_mask
                
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                opt.step()
            
            for batch in self.loaders['retain_train']:
                ids = batch['input_ids'].to(self.device)
                mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                out = self.model(ids, attention_mask=mask, labels=labels)
                with torch.no_grad():
                    ref_out = self.ref_model(ids, attention_mask=mask)
                
                kl_loss = self._kl_loss(out.logits, ref_out.logits)
                total_loss = out.loss + 0.7 * kl_loss
                
                opt.zero_grad()
                total_loss.backward()
                opt.step()
        
        return self.model


# ============================================================================
# Data Loading and Experiment Runner
# ============================================================================

def load_data(config, tokenizer):
    raw = load_dataset("ag_news")
    data = defaultdict(lambda: defaultdict(list))
    for split in ['train', 'test']:
        for item in raw[split]:
            data[split][item['label']].append(item['text'])
    return data


def make_loader(texts, labels, tokenizer, batch_size, shuffle=False):
    ds = TextClassificationDataset(texts, labels, tokenizer, 128)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)


def run_comprehensive_benchmark():
    conf = ExperimentConfig()
    ReportStyler.print_banner("Comprehensive SOTA Benchmark - Top 5 Methods")
    
    print(f"\n{ReportStyler.BOLD}Configuration:{ReportStyler.ENDC}")
    print(f"  Device: {conf.DEVICE}")
    print(f"  Model: {conf.MODEL_NAME}")
    print(f"  Seeds: {conf.SEEDS}")
    print(f"  Forget Scenarios: {conf.FORGET_SCENARIOS}")
    print(f"\n{ReportStyler.BOLD}Top 5 SOTA Methods:{ReportStyler.ENDC}")
    print(f"  1. GCM      - Gradient Conflict Minimization (H=0.9574)")
    print(f"  2. PPGA     - PCA-Protected Gradient Ascent (H=0.9565)")
    print(f"  3. ADU      - Alternating Direction Unlearning (H=0.9547)")
    print(f"  4. CPP      - Critical Path Preservation (H=0.9538)")
    print(f"  5. HFGCM    - Hybrid FFN + Gradient Conflict (H=0.9520)")
    
    tokenizer = AutoTokenizer.from_pretrained(conf.MODEL_NAME)
    raw_data = load_data(conf, tokenizer)
    evaluator = Evaluator(conf.DEVICE, conf)

    final_results = []
    
    # Exactly like baseline: iterate seeds → classes → scenarios
    for seed in conf.SEEDS:
        set_seed(seed)
        
        for f_cls in range(conf.NUM_CLASSES):
            class_names = ["World", "Sports", "Business", "Sci/Tech"]
            target_name = class_names[f_cls]
            
            for n_forget in conf.FORGET_SCENARIOS:
                ReportStyler.print_scenario_header(target_name, n_forget, seed)
                
                # Prepare data
                f_train_txt = raw_data['train'][f_cls][:n_forget]
                f_train_lbl = [f_cls] * n_forget
                
                r_train_txt, r_train_lbl = [], []
                for c in range(4):
                    if c != f_cls:
                        r_train_txt.extend(raw_data['train'][c][:conf.RETAIN_SAMPLES_PER_CLASS])
                        r_train_lbl.extend([c] * conf.RETAIN_SAMPLES_PER_CLASS)
                
                t_f_txt = raw_data['test'][f_cls][:200]
                t_f_lbl = [f_cls] * 200
                t_r_txt, t_r_lbl = [], []
                for c in range(4):
                    if c != f_cls:
                        t_r_txt.extend(raw_data['test'][c][:200])
                        t_r_lbl.extend([c] * 200)

                loaders = {
                    'forget_train': make_loader(f_train_txt, f_train_lbl, tokenizer, conf.BATCH_SIZE, True),
                    'retain_train': make_loader(r_train_txt, r_train_lbl, tokenizer, conf.BATCH_SIZE, True),
                    'forget_test': make_loader(t_f_txt, t_f_lbl, tokenizer, conf.BATCH_SIZE, False),
                    'retain_test': make_loader(t_r_txt, t_r_lbl, tokenizer, conf.BATCH_SIZE, False),
                    'test_all': make_loader(t_f_txt + t_r_txt, t_f_lbl + t_r_lbl, tokenizer, conf.BATCH_SIZE, False)
                }

                # Pre-trained baseline
                model = AutoModelForSequenceClassification.from_pretrained(conf.MODEL_NAME, num_labels=4).to(conf.DEVICE)
                base_metrics = evaluator.get_full_metrics(model, loaders, f_cls)
                ReportStyler.print_baseline_row("PRE-TRAINED", base_metrics['f_acc'], base_metrics['r_acc'], base_metrics['all_acc'], base_metrics['ece'], base_metrics['mia_auc'])

                # Fine-tuning
                model.train()
                opt = torch.optim.AdamW(model.parameters(), lr=conf.LEARNING_RATE)
                combined = ConcatDataset([loaders['forget_train'].dataset, loaders['retain_train'].dataset])
                train_l = DataLoader(combined, batch_size=conf.BATCH_SIZE, shuffle=True)
                
                for _ in range(conf.FINETUNE_EPOCHS):
                    for batch in train_l:
                        out = model(batch['input_ids'].to(conf.DEVICE), attention_mask=batch['attention_mask'].to(conf.DEVICE), labels=batch['labels'].to(conf.DEVICE))
                        opt.zero_grad()
                        out.loss.backward()
                        opt.step()
                
                ft_metrics = evaluator.get_full_metrics(model, loaders, f_cls)
                ReportStyler.print_baseline_row("FINE-TUNED", ft_metrics['f_acc'], ft_metrics['r_acc'], ft_metrics['all_acc'], ft_metrics['ece'], ft_metrics['mia_auc'])
                
                if ft_metrics['f_acc'] < 0.7:
                    print(f"{ReportStyler.FAIL}Skipping: Model failed to learn target class.{ReportStyler.ENDC}")
                    del model
                    clear_memory()
                    continue

                # Unlearning with Top 5 methods
                ReportStyler.print_table_header()
                
                methods = [
                    ("GCM", GradientConflictMinimization),
                    ("PPGA", PCAProtectedGradientAscent),
                    ("ADU", AlternatingDirectionUnlearning),
                    ("CPP", CriticalPathPreservation),
                    ("HFGCM", HybridFFNGradientConflict),
                ]

                ft_model_cpu = safe_deepcopy_model(model).to('cpu')

                for name, Cls in methods:
                    try:
                        current_model = safe_deepcopy_model(ft_model_cpu).to(conf.DEVICE)
                        
                        t0 = time.time()
                        unlearner = Cls(current_model, conf.DEVICE, loaders, conf, f_cls)
                        unlearned_model = unlearner.run()
                        dur = time.time() - t0
                        
                        res = evaluator.get_full_metrics(unlearned_model, loaders, f_cls)
                        
                        ReportStyler.print_result_row(
                            name, res['f_acc'], res['r_acc'], res['all_acc'],
                            res['ece'], res['mia_auc'], res['privacy_score'],
                            res['h_score'], dur
                        )
                        
                        final_results.append({
                            'seed': seed, 'class': target_name, 'n': n_forget, 'method': name,
                            **res, 'time': dur
                        })
                        
                        del unlearned_model, unlearner, current_model
                        clear_memory()
                        
                    except Exception as e:
                        print(f"│ {ReportStyler.FAIL}{name:<10} │ ERROR: {str(e)[:50]}{ReportStyler.ENDC}")
                        import traceback
                        traceback.print_exc()

                print(f"│ {'─'*138}")
                del model, ft_model_cpu
                clear_memory()

    # ========================================================================
    # FINAL STATISTICAL SUMMARY (Like baseline)
    # ========================================================================
    
    print("\n" + "=" * 140)
    print(f"{ReportStyler.BOLD}COMPREHENSIVE BENCHMARK COMPLETE{ReportStyler.ENDC}")
    print("=" * 140)
    
    if final_results:
        # Aggregate statistics
        method_stats = defaultdict(lambda: {'f_acc': [], 'r_acc': [], 'h_score': [], 'privacy': [], 'time': []})
        
        for r in final_results:
            m = r['method']
            method_stats[m]['f_acc'].append(r['f_acc'])
            method_stats[m]['r_acc'].append(r['r_acc'])
            method_stats[m]['h_score'].append(r['h_score'])
            method_stats[m]['privacy'].append(r['privacy_score'])
            method_stats[m]['time'].append(r['time'])
        
        print(f"\n{ReportStyler.BOLD}AGGREGATED RESULTS (Mean ± Std across all seeds/classes/scenarios):{ReportStyler.ENDC}")
        print(f"\n{'Method':<10} │ {'F-Acc':<16} │ {'R-Acc':<16} │ {'H-Score':<16} │ {'Privacy':<16} │ {'Time':<10}")
        print("-" * 100)
        
        ranking = []
        for method, stats in sorted(method_stats.items()):
            f_mean, f_std = np.mean(stats['f_acc']), np.std(stats['f_acc'])
            r_mean, r_std = np.mean(stats['r_acc']), np.std(stats['r_acc'])
            h_mean, h_std = np.mean(stats['h_score']), np.std(stats['h_score'])
            p_mean, p_std = np.mean(stats['privacy']), np.std(stats['privacy'])
            t_mean = np.mean(stats['time'])
            
            print(f"{method:<10} │ {f_mean:.3f}±{f_std:.3f}      │ {r_mean:.3f}±{r_std:.3f}      │ {h_mean:.3f}±{h_std:.3f}      │ {p_mean:.3f}±{p_std:.3f}      │ {t_mean:.1f}s")
            ranking.append((method, h_mean, f_mean, r_mean, p_mean, t_mean))
        
        ranking.sort(key=lambda x: -x[1])
        
        print("\n" + "=" * 140)
        print(f"{ReportStyler.BOLD}FINAL RANKING (by mean H-Score):{ReportStyler.ENDC}")
        print("=" * 140)
        for rank, (method, h, f, r, p, t) in enumerate(ranking, 1):
            print(f"  {rank}. {method:<10} H={h:.4f} | F={f:.3f} | R={r:.3f} | Privacy={p:.3f} | Time={t:.1f}s")
        
        print("\n" + "=" * 140)
        print(f"{ReportStyler.BOLD}PAPER-READY LATEX TABLE:{ReportStyler.ENDC}")
        print("=" * 140)
        print("\\begin{table}[h]")
        print("\\centering")
        print("\\caption{Top 5 SOTA Unlearning Methods - Comprehensive Benchmark}")
        print("\\begin{tabular}{l|cccc}")
        print("\\hline")
        print("Method & F-Acc $\\downarrow$ & R-Acc $\\uparrow$ & H-Score $\\uparrow$ & Time (s) \\\\")
        print("\\hline")
        for method, h, f, r, p, t in ranking:
            print(f"{method} & {f:.3f} & {r:.3f} & \\textbf{{{h:.4f}}} & {t:.1f} \\\\")
        print("\\hline")
        print("\\end{tabular}")
        print("\\end{table}")
        
        print("\n" + "=" * 140)

    return final_results


if __name__ == "__main__":
    results = run_comprehensive_benchmark()


  COMPREHENSIVE SOTA BENCHMARK - TOP 5 METHODS

Configuration:
  Device: cuda
  Model: distilbert-base-uncased
  Seeds: [42, 123, 456]
  Forget Scenarios: [50, 100, 200]

Top 5 SOTA Methods:
  1. GCM      - Gradient Conflict Minimization (H=0.9574)
  2. PPGA     - PCA-Protected Gradient Ascent (H=0.9565)
  3. ADU      - Alternating Direction Unlearning (H=0.9547)
  4. CPP      - Critical Path Preservation (H=0.9538)
  5. HFGCM    - Hybrid FFN + Gradient Conflict (H=0.9520)

┌── SCENARIO: Forget 'World' | N=50 | Seed=42 ────────────────────────────────────────────────────────────────────────────────
│ PRE-TRAINED  │ F-Acc: 0.000 │ R-Acc: 0.308 │ All-Acc: 0.231 │ ECE: 0.034 │ MIA: 0.378
│ FINE-TUNED   │ F-Acc: 0.615 │ R-Acc: 0.868 │ All-Acc: 0.805 │ ECE: 0.065 │ MIA: 0.196
Skipping: Model failed to learn target class.

┌── SCENARIO: Forget 'World' | N=100 | Seed=42 ────────────────────────────────────────────────────────────────────────────────
│ PRE-TRAINED  │ F-Acc: 0.005 │ R-Acc: 0.3